<a href="https://colab.research.google.com/github/lucasflores/PermaLost/blob/main/PermaFrostProj.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#Permafrost Project

## The Problem

Melty melt leads to decay of 'soft' artifacts -> race to save these from decay/rot

## The Project

In this project I will use permafrost data provided by the European Space Agency's (ESA) Climate Change Initiative (CCI) Permafrost project.  It is derived from a thermal model driven and constrained by satellite data.

## Data Types
* **GTD** Ground temp (by depth)
* **PFR** Permafrost extent
* **ALT** Active Layer Thickness

This corresponds to average annual ground temperatures and is provided for specific depths (surface, 1m, 2m, 5m , 10m)

Active Layer Thickness is the thickness of the layer of the ground that is subject to annual thawing and freezing in areas underlain by permafrost. The thickness of the active layer depends on such factors as the ambient air temperature, vegetation, drainage, soil or rock type and total water content, snowcover, and degree and orientation of slope. As a rule, the active layer is thin in the High Arctic (it can be less than 15 cm) and becomes thicker farther south (1 m or more).
The thickness of the active layer can vary from year to year, primarily due to variations in the mean annual air temperature, distribution of soil moisture, and snowcover
plotHistograms.py

Permafrost is an Essential Climate Variable (ECV) within the Global Climate Observing System (GCOS), which is characterized by subsurface temperatures and the depth of the seasonal thaw layer. Complementing ground-based monitoring networks, the Permafrost CCI project is establishing Earth Observation (EO) based products for the permafrost ECV spanning the last two decades. Since ground temperature and thaw depth cannot be directly observed from space-borne sensors, a variety of satellite and reanalysis data are combined in a ground thermal model. The algorithm uses remotely sensed data sets of Land Surface Temperature (MODIS LST/ ESA LST CCI) and landcover (ESA Landcover CCI) to drive the transient permafrost model CryoGrid CCI, which yields thaw depth and ground temperature at various depths, while ground temperature forms the basis for permafrost fraction.



### Project Package Dependencies  
- pandas 
- geopandas 
- matplotlib

### ML/Analysis algorithm note
* Geographic Weighted Regression seems to be a reasonable start for some predictive analysis via the mgwr python package (more details here https://deepnote.com/@carlos-mendez/PYTHON-GWR-and-MGWR-cd2LqaPqTSibIEHMiigreg)
* Might just be useful for interpolation. Not sure if I can do like a time series regression. 
* gwr/mgwr has a “bandwidth“ parameter which sets the “neighborhood” size to be used for the location based regression. Also it seems to apply some shooting which I should read up more on.  

## General Notes
### *04/05/22* 
* Data downloading, combing of separate PFR, ATL, and GTD files, and deleting nan and non Greenland data in dfs (most expensive/longest steps) takes about 1-1.5 hours.
* Downloads faster on google cola but not enough RAM to do row dropping computations. Trying to find smart ways to release RAM. Should check in indices of “”Greenland” lat/longs exist in some relatively close range that I can take advantage of so I don’t have to loop through every goddamn instance. 
* Want to implement a memory profiler as well. 

In [ ]:
%%time 

# Important library for many geopython libraries
#!apt install gdal-bin python-gdal python3-gdal 
# Install rtree - Geopandas requirment
#!apt install python3-rtree 
# Install Geopandas
!pip install --upgrade geopandas
# Install descartes - Geopandas requirment
#!pip install descartes 
# Install Folium for Geographic data visualization
#!pip install folium
# Install plotlyExpress
#!pip install plotly_expres
#!pip install mapclassify
#!pip install xarray
!pip install folium matplotlib mapclassify
!pip3 install pickle5
!pip install wget

In [3]:
import os
import pandas as pd
import sklearn
import numpy as np
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
#import pickle5 as pickle
import geopandas as gpd
from shapely.geometry import Point
import folium
import wget
import xarray as xr
import gc   #garbage collector interface

# Get The Data

In [4]:
gc.collect()

350

In [7]:
def huge_intermediate_calc(dfPFR):
  dfPFR.drop(dfPFR[dfPFR['x'] < -2500000.0].index, inplace=True)
  print('poop55')
  print('poop22')
  print('poop45')
  print('poop23')
  dfPFR.drop(dfPFR[dfPFR['x'] > 4000000.0].index, inplace=True)
  return dfPFR

import multiprocessing
import argparse

#for year in ['1997','1998','1999']:
#parser = argparse.ArgumentParser(description="%prog [options]", formatter_class=argparse.ArgumentDefaultsHelpFormatter)
#parser.add_argument("-selectGreenland_to_pickle", dest='selectGreenland_to_pickle', action='store_true', default=False, help="")
#parser.add_argument("-havePickle", dest='havePickle', action='store_true', default=False, help="")
#args = parser.parse_args()

selectGreenland_to_pickle=False

if selectGreenland_to_pickle :
  for year in ['1998']:
    ALT = "https://dap.ceda.ac.uk/neodc/esacci/permafrost/data/active_layer_thickness/L4/area4/pp/v03.0//ESACCI-PERMAFROST-L4-ALT-ERA5_MODISLST_BIASCORRECTED-AREA4_PP-"+year+"-fv03.0.nc"
    PFR = "https://dap.ceda.ac.uk/neodc/esacci/permafrost/data/permafrost_extent/L4/area4/pp/v03.0//ESACCI-PERMAFROST-L4-PFR-ERA5_MODISLST_BIASCORRECTED-AREA4_PP-"+year+"-fv03.0.nc"
    GTD = "https://dap.ceda.ac.uk/neodc/esacci/permafrost/data/ground_temperature/L4/area4/pp/v03.0//ESACCI-PERMAFROST-L4-GTD-ERA5_MODISLST_BIASCORRECTED-AREA4_PP-"+year+"-fv03.0.nc"
  
    wget.download(PFR, './')
    dsPFR = xr.open_dataset('ESACCI-PERMAFROST-L4-PFR-ERA5_MODISLST_BIASCORRECTED-AREA4_PP-'+year+'-fv03.0.nc')
    os.remove('ESACCI-PERMAFROST-L4-PFR-ERA5_MODISLST_BIASCORRECTED-AREA4_PP-'+year+'-fv03.0.nc')
    dfPFR = dsPFR.to_dataframe()
    del dsPFR
    %reset Out
    gc.collect() #collect the memory garbage
    dfPFR = dfPFR.reset_index()
    # Restrict to Lat/Long containing Greenland only
    #dfPFR = multiprocessing.Pool(1).map(huge_intermediate_calc, [dfPFR])[0]
    dfPFR.drop(dfPFR[dfPFR['x'] < -2500000.0].index, inplace=True)
    dfPFR.drop(dfPFR[dfPFR['x'] > -274000.0].index, inplace=True)
    dfPFR.drop(dfPFR[dfPFR['y'] < -2900000.0].index, inplace=True)
    dfPFR.drop(dfPFR[dfPFR['y'] > -420000.0].index, inplace=True)
  
    wget.download(ALT, './')
    dsALT = xr.open_dataset('ESACCI-PERMAFROST-L4-ALT-ERA5_MODISLST_BIASCORRECTED-AREA4_PP-'+year+'-fv03.0.nc')
    os.remove('ESACCI-PERMAFROST-L4-ALT-ERA5_MODISLST_BIASCORRECTED-AREA4_PP-'+year+'-fv03.0.nc')
    dfALT = dsALT.to_dataframe()
    del dsALT
    dfALT = dfALT.reset_index()
    dfALT = dfALT.drop(dfALT[dfALT['x'] < -2500000.0].index)
    dfALT = dfALT.drop(dfALT[dfALT['x'] > -274000.0].index)
    dfALT = dfALT.drop(dfALT[dfALT['y'] < -2900000.0].index)
    dfALT = dfALT.drop(dfALT[dfALT['y'] > -420000.0].index)
  
    wget.download(GTD, './')
    dsGTD = xr.open_dataset('ESACCI-PERMAFROST-L4-GTD-ERA5_MODISLST_BIASCORRECTED-AREA4_PP-'+year+'-fv03.0.nc')
    os.remove('ESACCI-PERMAFROST-L4-GTD-ERA5_MODISLST_BIASCORRECTED-AREA4_PP-'+year+'-fv03.0.nc')
    dfGTD = dsGTD.to_dataframe()
    del dsGTD
    dfGTD = dfGTD.reset_index()
    dfGTD = dfGTD.drop(dfGTD[dfGTD['x'] < -2500000.0].index)
    dfGTD = dfGTD.drop(dfGTD[dfGTD['x'] > -274000.0].index)
    dfGTD = dfGTD.drop(dfGTD[dfGTD['y'] < -2900000.0].index)
    dfGTD = dfGTD.drop(dfGTD[dfGTD['y'] > -420000.0].index)
  
    print('poop0')
  
    dfGreenland = pd.concat([dfPFR, dfALT['ALT'], dfGTD['GST'], dfGTD['T1m'], dfGTD['T2m'], dfGTD['T5m'], dfGTD['T10m'] ], axis=1)
    del dfPFR
    del dfALT
    del dfGTD
    dfGreenland = dfGreenland.dropna(subset=['ALT','GST','T1m','T2m','T5m','T10m'], how='all')
    dfGreenland.to_pickle("Greenland_noNANallColumns_"+year+"_Pickle.pkl")
    gdfGreenland = gpd.GeoDataFrame( dfGreenland, geometry=gpd.points_from_xy(dfGreenland.x, dfGreenland.y))

%ls

sample_data/


In [ ]:
%ls

In [ ]:
# Upload Greenland_noNANallColumns_2018_Pickle.pkl here
from google.colab import files
uploaded = files.upload()

In [ ]:
with open('Greenland_noNANallColumns_1997_Pickle.pkl', 'rb') as pickle_file:
    dfGreenland = pickle.load(pickle_file)

In [ ]:
dfGreenland.head()
# Convert DataFrame to a geoDataFrame
gdfGreenland = gpd.GeoDataFrame( dfGreenland, geometry=gpd.points_from_xy(dfGreenland.x, dfGreenland.y))

In [ ]:
gdfGreenland.head()

In [ ]:
#world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
world = gpd.read_file('jf791qw8489.shp')
world.head()
#world = world[(world.name != "Antarctica") & (world.name != "Fr. S. Antarctic Lands") & (world.name == "Greenland")]

world = world.to_crs("EPSG:3995") # world.to_crs(epsg=3395) would also work
gdfGreenland.crs = "EPSG:3995"
ax = world.plot(figsize=(24,18))
ax.set_title("Greenland")
ax.set_ylim([-2.2e6, -1.9e6])
ax.set_xlim([-1.1e6, -0.8e6])

gdfGreenland_epsg4326 = gdfGreenland.to_crs("EPSG:4326")

#world.explore()

# We can now plot our ``GeoDataFrame``.
gdfGreenland.plot(ax=ax, column='PFR', markersize=0.5, cmap="plasma", vmin=45.0, legend=True, alpha=0.7)


#gdfGreenland.explore("area", legend=False)
#m= folium.Map(location = [75.0,-41.0], tiles = "Stamen Terrain", zoom_start = 9)
#gdfGreenland_epsg4326.explore(column='PFR', m=m)



plt.show()

In [ ]:
map = folium.Map(location = [13.406,80.110], tiles = "Stamen Terrain", zoom_start = 9)
map

In [ ]:
gdfGreenland.head()
gdfGreenland.plot(ax=ax, column='PFR', markersize=0.5, alpha=0.7)
plt.show()

In [ ]:
print(gdfGreenland.crs)